In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [3]:
port_train_df = pd.read_csv('train.csv')
port_test_df = pd.read_csv('test.csv')

In [4]:
all_data = pd.concat((port_train_df.loc[:,'portfolio_id':'status'],
                      port_test_df.loc[:,'portfolio_id':'status']))

In [5]:
from datetime import datetime

all_data['start_date']=all_data['start_date'].apply(lambda x: "{0:-08d}".format(x))

all_data['start_date']=all_data['start_date'].apply(lambda x: datetime.strptime(x,'%Y%m%d'))

In [6]:
all_data['sell_date']=all_data['sell_date'].apply(lambda x: "{0:-08d}".format(x))
all_data['sell_date']=all_data['sell_date'].apply(lambda x: datetime.strptime(x,'%Y%m%d'))

In [7]:

all_data['creation_date']=all_data['creation_date'].apply(lambda x: "{0:-08d}".format(x))
all_data['creation_date']=all_data['creation_date'].apply(lambda x: datetime.strptime(x,'%Y%m%d'))

In [11]:
all_data.dtypes

portfolio_id              object
desk_id                   object
office_id                 object
pf_category               object
start_date        datetime64[ns]
sold                     float64
country_code              object
euribor_rate             float64
currency                  object
libor_rate               float64
bought                   float64
creation_date     datetime64[ns]
indicator_code            object
sell_date         datetime64[ns]
type                      object
hedge_value               object
status                    object
dtype: object

In [9]:
from scipy.stats import skew
#get all the columns with numeric data types
numeric_feats =all_data.dtypes[(all_data.dtypes!='object') & (all_data.dtypes!='datetime64[ns]')].index

print numeric_feats

Index([u'sold', u'euribor_rate', u'libor_rate', u'bought'], dtype='object')


In [12]:
all_data[numeric_feats].isnull().sum()

sold              2
euribor_rate      0
libor_rate      739
bought            2
dtype: int64

In [13]:
#missing value are there first impute it 
all_data["libor_rate"]=all_data["libor_rate"].fillna(all_data["libor_rate"].median())
#fill missing value for bought
all_data["bought"]=all_data["bought"].fillna(all_data["bought"].median())

all_data["sold"].fillna(all_data["sold"].median(), inplace=True)

In [14]:
#standarized the data 
# standardize the data attributes
all_data[numeric_feats] = preprocessing.scale(all_data[numeric_feats])

C:\Users\Ravi\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:160: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [15]:
all_data.head(10)

,portfolio_id,desk_id,office_id,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,bought,creation_date,indicator_code,sell_date,type,hedge_value,status
0,PF00001002,DSK00001001,OFF00001002,B,2004-07-20,0.044765,T,0.883079,USD,0.985558,0.044158,2004-07-20,NaN,2004-08-12,B,NaN,NaN
1,PF00001003,DSK00001002,OFF00001001,A,2004-07-09,0.424661,N,0.883079,GBP,3.083167,0.421505,2004-07-23,NaN,2004-08-12,C,NaN,NaN
2,PF00001005,DSK00001004,OFF00001001,A,2004-07-23,-0.260230,T,0.883079,USD,0.985558,-0.260404,2004-07-23,NaN,2004-08-17,A,NaN,NaN
3,PF00001006,DSK00001005,OFF00001001,A,2004-06-09,0.357094,T,0.883079,USD,0.985558,0.355955,2004-07-23,NaN,2004-07-13,A,NaN,NaN
4,PF00001007,DSK00001005,OFF00001002,B,2004-06-09,0.220266,T,0.883079,USD,0.985558,0.219328,2004-07-23,NaN,2004-07-13,B,NaN,NaN
5,PF00001008,DSK00001006,OFF00001001,A,2004-07-07,-0.304293,T,0.883079,USD,0.985558,-0.304457,2004-07-26,NaN,2004-08-10,A,NaN,NaN
6,PF00001010,DSK00001009,OFF00001001,A,2004-07-06,-0.236757,T,0.883079,USD,0.985558,-0.237020,2004-07-26,NaN,2004-08-09,A,NaN,NaN
7,PF00001011,DSK00001009,OFF00001002,B,2004-07-06,0.182658,T,0.883079,USD,0.985558,0.181778,2004-07-26,NaN,2004-08-09,B,NaN,NaN
8,PF00001012,DSK00001010,OFF00001001,A,2004-04-19,-0.109488,T,0.883079,USD,0.985558,-0.111477,2004-07-26,NaN,2004-07-20,A,NaN,NaN
9,PF00001016,DSK00001014,OFF00001001,A,2004-04-14,0.628681,T,0.883079,USD,0.985558,0.623324,2004-07-27,NaN,2004-07-13,A,NaN,NaN


In [16]:
#calculate duration
#all the data has been converted into datetime now we can calculate the duration

all_data['duration']=(all_data['creation_date']-all_data['start_date']).apply(lambda x:x.days)

In [17]:

all_data["hedge_value"]=all_data["hedge_value"].fillna(all_data["hedge_value"].median())

In [24]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#label encoder
objcolname=['pf_category','currency','type','country_code']
for col in objcolname:
    all_data[col]=le.fit_transform(all_data[col])

In [25]:
all_new_data=all_data.drop(['start_date', 'creation_date', 'sell_date', 'indicator_code', 'status', 'desk_id','portfolio_id','office_id'],axis=1)

In [26]:
X_train = all_new_data[:port_train_df.shape[0]]

In [27]:
y=port_train_df[['return']]

In [28]:
X_train.dtypes

pf_category       int64
sold            float64
country_code      int64
euribor_rate    float64
currency          int64
libor_rate      float64
bought          float64
type              int64
hedge_value      object
duration          int64
dtype: object

In [29]:
from sklearn import linear_model  
model = linear_model.LinearRegression()  
model.fit(X_train, y)  

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [30]:


scores = cross_val_score(model, X_train, y, scoring='r2', cv=5)
print(scores)
print('mean r2:',np.mean(scores))

NameError: name 'cross_val_score' is not defined